In [10]:
#pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o
import h2o
from h2o.automl import H2OAutoML
import os
import pandas as pd
import xlsxwriter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
h2o.init()
#h2o.init(ip="127.0.0.1",max_mem_size_GB = 4)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,21 mins 30 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.4
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_rosa_ijmsvh
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.947 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [3]:
#https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

In [4]:
# # Import a sample binary outcome train/test set into H2O
# train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
# test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")
# train.shape
# # Identify predictors and response
# x = train.columns
# y = "response"
# x.remove(y)
# # For binary classification, response should be a factor
# train[y] = train[y].asfactor()
# test[y] = test[y].asfactor()
# # Run AutoML for 20 base models
# aml = H2OAutoML(max_models=3, seed=1)
# aml.train(x=x, y=y, training_frame=train)
# # View the AutoML Leaderboard
# lb = aml.leaderboard
# lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

# # To generate predictions on a test set, you can make predictions
# # directly on the `H2OAutoML` object or on the leader model
# # object directly
# preds = aml.predict(test)

# # or:
# preds = aml.leader.predict(test)

In [5]:
# path_response='/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/pythoncode/Main Data Files/response/'
# for file in os.listdir(path_response):
#     response = h2o.import_file(path_response+file)
#     respnse=response.set_names(['C1', 'Variety2','y_c','y_b'])
#     #respnse_col = response.columns
#     #respnse_col.remove(respnse_col[1])
#     #respnse_col.remove(respnse_col[1])
    
#     path_x = '/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/pythoncode/Main Data Files/normalized_data/'
#     for file_folder in os.listdir(path_x):
#         if (file_folder[-4:] != '.csv') & (file_folder != '.DS_Store'):
#             path = path_x+file_folder
#             os.chdir(path)
#             file_list = []
#             for file in os.listdir(path):
#                 if (file[0] != 'R') & (file[-4:] == '.csv'):
#                     file_list.append(file) 
#                     data_temp = h2o.import_file(file) 
#                     data.merge(response)

In [23]:
path_response='/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/python code local/Main Data Files/response/'
for file_response in os.listdir(path_response):
    if (file_response != '.DS_Store')& (file_response != 'Icon\r'):
        print(file_response)
        path_r= path_response+file_response
        os.chdir(path_r)
        for re in os.listdir(path_r):
            if re[0:8] == 'response':
                response = pd.read_csv(path_response+file_response+'/'+re)
                response.rename(columns={'Column1':'Link_ID','x1':'y_b'}, inplace=True)
                response.drop(columns=response.columns[2], inplace=True)
                response=response.drop(columns='Variety2')
                path_x = '/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/python code local/Main Data Files/OTUData-1-1/'
                writer= pd.ExcelWriter(path_r+'/'+'classification_H2o'+'.xlsx', engine='xlsxwriter')
                for file_folder in os.listdir(path_x):
                    if (file_folder[-4:] != '.csv') & (file_folder != '.DS_Store')& (file_response != 'Icon\r'):
                        
                        path = path_x+file_folder
                        os.chdir(path)
                        file_list = []
                        classification=pd.DataFrame()
                        writer_feature = pd.ExcelWriter(path_r+'/'+'feature_important_H20'+'_'+file_folder+'.xlsx', engine='xlsxwriter')
                        for file in os.listdir(path):
                            if (file[0] != 't') & (file[-4:] == '.csv') & (file != '.DS_Store')& (file_response != 'Icon\r'):
                                print(file)
                                file_list.append(file)
                                data_temp = pd.read_csv(file)
                                data_temp.rename(columns={'Unnamed: 0':'Link_ID'}, inplace=True)
                                data=pd.merge(response,data_temp,on='Link_ID')
                                data.drop(columns="Link_ID",inplace=True)
                                train_pd,test_pd=train_test_split(data,train_size=0.8, random_state=42)
                                train_pd.to_csv('train.csv',index=False)
                                test_pd.to_csv('test.csv',index=False)
                                train= h2o.import_file('train.csv')
                                test= h2o.import_file('test.csv')
                                # Identify predictors and response
                                x = train.columns
                                y = "y_b"
                                x.remove(y)
                                # For binary classification, response should be a factor
                                train[y] = train[y].asfactor()
                                test[y] = test[y].asfactor()
                                # Run AutoML for 20 base models
                                aml = H2OAutoML(max_runtime_secs = 70, seed=1)#max_runtime_secs = 3600)#max_models=20
                                aml.train(x=x, y=y, training_frame=train)
                                # View the AutoML Leaderboard
                                lb = aml.leaderboard
                                m = aml.get_best_model(criterion="logloss")
                                print(m.params.keys())
                                # Get leaderboard with all possible columns
                                lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
                                print(lb)
                                #test=test.drop("y_b",axis=1)
                                if m is not None:
                                    preds = m.predict(test)    
                                    y_valid = preds['predict']
                                    report_RF = classification_report(test_pd['y_b'].values, y_valid.as_data_frame(),output_dict=True)
                                    dRF = pd.DataFrame(report_RF).transpose()
                                    classification=pd.concat([classification,dRF['f1-score']],axis=1)
                                    if m.varimp(use_pandas=True) is not None:
                                        m.varimp(use_pandas=True).to_excel(writer_feature, sheet_name=file, index=True)
                        writer_feature.save()   
                        classification.index=dRF.index
                        classification.to_excel(writer, sheet_name=file_folder, index=True)
                writer.save() 
                    
                       

no_tuber_silver_scurf
 1 _ 1 .csv
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |██
19:03:22.87: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 171.0.

█████████████████████████████ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_ac39f7ea3e4fdff39a8dfc6f8db34790> was cancelled by the user.

In [24]:
xgb = aml.get_best_model(algorithm="xgboost")

In [25]:
xgb.params.keys()

dict_keys(['model_id', 'training_frame', 'validation_frame', 'nfolds', 'keep_cross_validation_models', 'keep_cross_validation_predictions', 'keep_cross_validation_fold_assignment', 'score_each_iteration', 'fold_assignment', 'fold_column', 'response_column', 'ignored_columns', 'ignore_const_cols', 'offset_column', 'weights_column', 'stopping_rounds', 'stopping_metric', 'stopping_tolerance', 'max_runtime_secs', 'seed', 'distribution', 'tweedie_power', 'categorical_encoding', 'quiet_mode', 'checkpoint', 'export_checkpoints_dir', 'ntrees', 'max_depth', 'min_rows', 'min_child_weight', 'learn_rate', 'eta', 'sample_rate', 'subsample', 'col_sample_rate', 'colsample_bylevel', 'col_sample_rate_per_tree', 'colsample_bytree', 'colsample_bynode', 'max_abs_leafnode_pred', 'max_delta_step', 'monotone_constraints', 'interaction_constraints', 'score_tree_interval', 'min_split_improvement', 'gamma', 'nthread', 'save_matrix_directory', 'build_tree_one_node', 'calibrate_model', 'calibration_frame', 'max_b

In [26]:
# Inspect individual parameter values
xgb.params['ntrees']

{'default': 50, 'actual': 39, 'input': 10000}

In [12]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")

In [15]:
m.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_4_AutoML_5_20220826_184804',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_4_AutoML_5_20220826_184804'},
  'input': None},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'AutoML_5_20220826_184804_training_py_30_sid_8bd2',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/AutoML_5_20220826_184804_training_py_30_sid_8bd2'},
  'input': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'AutoML_5_20220826_184804_training_py_30_sid_8bd2',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/AutoML_5_20220826_184804_training_py_30_sid_8bd2'}},
 'validation_frame': {'default': None, 'actual': None, 'input': None},
 'nfolds': {'default': 0, 'actual': 5, 'input': 5},
 'keep_c

In [16]:
m.get_params()

{'model_id': None,
 'training_frame': None,
 'validation_frame': None,
 'nfolds': 0,
 'keep_cross_validation_models': True,
 'keep_cross_validation_predictions': False,
 'keep_cross_validation_fold_assignment': False,
 'score_each_iteration': False,
 'score_tree_interval': 0,
 'fold_assignment': 'auto',
 'fold_column': None,
 'response_column': None,
 'ignored_columns': None,
 'ignore_const_cols': True,
 'offset_column': None,
 'weights_column': None,
 'balance_classes': False,
 'class_sampling_factors': None,
 'max_after_balance_size': 5.0,
 'max_confusion_matrix_size': 20,
 'ntrees': 50,
 'max_depth': 5,
 'min_rows': 10.0,
 'nbins': 20,
 'nbins_top_level': 1024,
 'nbins_cats': 1024,
 'r2_stopping': None,
 'stopping_rounds': 0,
 'stopping_metric': 'auto',
 'stopping_tolerance': 0.001,
 'max_runtime_secs': 0.0,
 'seed': -1,
 'build_tree_one_node': False,
 'learn_rate': 0.1,
 'learn_rate_annealing': 1.0,
 'distribution': 'auto',
 'quantile_alpha': 0.5,
 'tweedie_power': 1.5,
 'huber_alp

In [ ]:
m.

In [7]:
# from h2o.estimators import H2OGradientBoostingEstimator
# valid = test
# response_col = "y_b"
# distribution = "multinomial"
# gbm = H2OGradientBoostingEstimator(nfolds=3,
#                                    distribution=distribution)
# gbm.train(x=x,
#           y=response_col,
#           training_frame=train,
#           validation_frame=valid)
# confusion_matrix = gbm.confusion_matrix()
# confusion_matrix
#gbm.predict(test)

In [8]:
writer.save()

/Users/rosa/opt/anaconda3/envs/soil_env2/lib/python3.10/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
